<a href="https://colab.research.google.com/github/mamatqobulovsardor/Regressiya_ko-rsatkichlari/blob/main/Regressiya_ko'rsatkichlari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Regressiya Metrikalari

Ushbu daftarda biz eng keng tarqalgan regressiya metrikalarini ko‘rib chiqamiz. Bu 365 Data Science kursining ML jarayoniga bag‘ishlangan qo‘shimcha ish daftari hisoblanadi. Ushbu daftarda faqat implementatsiya qismi ko‘rib chiqiladi. Har bir yondashuvning batafsil tushuntirishlari uchun kursni yoki hujjatlarni ko‘rib chiqing

R2 Skori
Adj R2 Skori
O'rtacha Mutlaq Xato (MAE)
Kök O'rtacha Kvadrat Xato (RMSE)

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

##Ma'lumotlarni yuklash

Ushbu ma'lumotlar to'plami uy-joylar narxiga oid ma'lumotlardan iborat. Ma'lumotlar to'plamidagi xususiyatlarga asoslanib, uylarga optimal narxni belgilash uchun ML modellari keng qo'llaniladi. Ushbu daftarda biz uy narxini bashorat qilish modeli yaratib, uylarga narx belgilashga yordam beramiz

In [30]:
data = pd.read_csv("https://raw.githubusercontent.com/PlayingNumbers/ML_Process_Course/refs/heads/main/12_model_evaluation/housing_data.csv")

data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


##Xususiyatlarni Tanlash

Keling, modelda foydalanmoqchi bo'lgan xususiyatlarni tanlaylik. Ishni oddiy qilish uchun, biz qo'lda tanlangan xususiyatlar ro'yxatini ishlatamiz:

In [31]:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated']

target = "price"

##Kross-Validatsiya

Keyingi qadamda, ma'lumotlar to'plamimizni o'quv (training) va test (testing) to'plamlariga bo'lishimiz kerak:

In [36]:
from sklearn.model_selection import train_test_split

y = data[target]
X = data[features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

##Model

  Ishni oddiy qilish uchun, biz chiziqli regressiya modelini ishlatamiz:

In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)
y_preds = model.predict(X_test)

##Baholash Metrikalari

Eng keng tarqalgan baholash metrikalari — r2_score, rmse va mae. Bu yerda, sklearn o'zining implementatsiyasiga ega:

In [40]:
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error
)

r2 = r2_score(y_test, y_preds)
rmse = np.sqrt(mean_squared_error(y_test, y_preds))
mae = mean_absolute_error(y_test, y_preds)

print("r2_score: {0}".format(r2))
print("rmse: {0}".format(rmse))
print("mae:{0}".format(mae))

r2_score: 0.0662329312093658
rmse: 788277.1876925749
mae:191902.15686060506


## R2 Score

R-squaredning intuitiv tushunchasi shundaki, u bizga y o'zgaruvchisi bo'yicha bashorat xatosining necha foizi modelingiz tomonidan bartaraf etilganini yoki tushuntirilganini aytib beradi. Buni biz "moslikning yaxshiligi"ni aniqlash uchun ishlatamiz.

Mana, R2 ning ichki ishlashini ko'rishingiz uchun R2 ning implementatsiyasi:

In [41]:
def r2_score(y_test, y_preds):
    SS_reg = np.sum((y_test - y_preds)**2)
    SS_total = np.sum((y_test - np.mean(y_test))**2)
    r2 = 1-SS_reg/SS_total

    return r2

r2_score(y_test, y_preds)

0.0662329312093658

## Tuzatilgan R-Squared


Muammo shundaki, R-Squared osonlik bilan manipulyatsiya qilinishi mumkin. Agar biz modelimizni haddan tashqari moslashtirsak, bu har doim r2 skorini oshiradi. Shuning uchun yechim — tuzatilgan R-squaredni ishlatishdir. Tuzatilgan R-squared modelimizdagi xususiyatlar soniga asoslanib, R-squared raqamini moslashtiradi.

In [42]:
def adf_r2_score(X, y_test, y_preds):
    SS_reg = np.sum((y_test - y_preds)**2)
    SS_total = np.sum((y_test - np.mean(y_test))**2)
    r2 = 1-SS_reg/SS_total

    N = len(X)
    p = len(X.columns)

    adj_r2 = 1-((1-r2)*(N-1))/(N-p-1)
    return adj_r2

adj_r2_score(X, y_test, y_preds)

0.06379011350160735

##O'rtacha Mutlaq Xato


Oddiy qilib aytganda, biz har bir ma'lumot nuqtasi uchun mutlaq o'rtacha xatolarni ko'rib chiqamiz. So'ngra ularning o'rtachasini olishamiz. Bu bizga ma'lumotlar to'plamimizdagi o'rtacha xatoning kattaligini beradi.

In [43]:
def mean_absolute_error(y_test, y_preds):
    return np.sum(abs(y_preds - y_test))/len(y_preds)

mean_absolute_error(y_test, y_preds)

191902.15686060506

##Ko'k O'rtacha Kvadrat Xato (RMSE)


Xatoliklarning mutlaq qiymatini olish o'rniga, bu holatda avval xatoliklarni kvadratlashamiz. Bu barcha xatoliklarni musbat qilishni ta'minlaydi. So'ngra kvadratlangan xatoliklarning o'rtachasini olamiz, bu o'rtacha kvadrat xato (MSE) ga aylanadi. Nihoyat, biz ildiz olishimiz orqali RMSE ni hisoblaymiz

In [44]:
def mean_squared_error(y_test, y_preds):
    return np.sum((y_preds - y_test)**2)/len(y_preds)

np.sqrt(mean_squared_error(y_test, y_preds))

788277.1876925749